In [ ]:
# Checklist...
#
# Make sure units have been converted in fisher matrix to match units in mcmc chains (use 'Fisher_Units_Converter' notebook.)
#
# Make sure parameters in fisher matrix match order of parameter in MCMC
#
# Make sure you burn in the right number of points in the MCMC chains before plotting below 
#
# Choose where to plot Fisher contour centers (where you forecasted [correct choice], or max likelihood of MCMCs)
#
#

In [ ]:
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
# plt.rcParams['text.usetex']=True

In [ ]:
# First run montepython to determine the burn-in period 
# Specify either number of lines (e.g. 100) or proportion (e.g. 0.3)
yesstep_burnin = 100
nostep_burnin = 100

In [ ]:
#Import MCMC Chains

chains_dir = "/Users/nicholasdeporzio/Desktop/cfworkspace/chains/"
#yesstep_dir = chains_dir + "chains_yes_relicfast/chains_only/2019-12-04_1000000_"
#nostep_dir = chains_dir + "chains_no_relicfast/chains_only/2019-12-04_1000000_"
yesstep_dir = chains_dir + "chains_yes_relicfast/chains_only/2020-02-12_1000000_"
nostep_dir = chains_dir + "chains_no_relicfast/chains_only/2020-01-30_1000000_"

samples_yesstep = loadMCSamples(yesstep_dir, settings={'ignore_rows':yesstep_burnin})
samples_nostep = loadMCSamples(nostep_dir, settings={'ignore_rows':nostep_burnin})


In [ ]:
# Triangle plot, no Fisher Contours
g = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
g.settings.figure_legend_frame = False #Box around legend
g.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
#g.settings.scaling_factor = 0.3 #Scaling of font size
#g.settings.title_limit_fontsize = 14
g.settings.axes_fontsize=14
g.settings.axes_labelsize=20
g.settings.axis_tick_x_rotation=45
g.settings.legend_fontsize=20
#g.settings.axis_tick_y_rotation=90
#g.settings.colorbar_tick_rotation=90

g.triangle_plot(
    [samples_yesstep, samples_nostep], 
    filled=True,
    #legend_labels=['MCMC, DESI, Relic/LCDM Steps ON', 'MCMC, DESI, Relic/LCDM Steps OFF']
    legend_labels=['MCMC, EUCLID, Relic/LCDM Steps ON', 'MCMC, EUCLID, Relic/LCDM Steps OFF']
    )
plt.savefig("/Users/nicholasdeporzio/Desktop/only_mcmc.png")

In [ ]:
# Add Fisher Contours
# Need to adjust Fisher parameter ordering to MCMC parameter ordering
# DESI Fisher parameter ordering: omega_b, omega_cdm, n_s, A_s, tau_reio, h, m_0, sigma_fog, b0_bar, alpha_k2
# DESI MCMC parameter ordering: omega_b, omega_cdm, n_s, A_s, h, tau_reio, m_0, sigma_fog, b0_bar, alpha_k2

# EUCLID Fisher parameter ordering: omega_b, omega_cdm, n_s, A_s, tau_reio, h, m_0, sigma_fog, beta0, beta1, alpha_k2
# EUCLID MCMC parameter ordering: omega_b, omega_cdm, n_s, A_s, h, tau_reio, m_0, sigma_fog, beta0, beta1, alpha_k2

fisher_reorder_index = [0, 1, 2, 3, 5, 4, 6, 7, 8, 9, 10]

#yesstep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/DESI_MCMC_Match/ps2_yesrelicfast/inv_fisher.mat", skiprows=1)
#nostep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/DESI_MCMC_Match/ps2_norelicfast/inv_fisher.mat", skiprows=1)
#nostep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/desi_preshift_RelicStepsOff_fidsOnMcmcBestfitNoRelicStep/inv_fisher.mat", skiprows=1)

yesstep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/Neutrino_Match_Beta2/ps3_CMBS4_yesrelicfast_beta2bestfit/inv_fisher.mat", skiprows=1)
nostep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/Neutrino_Match_Beta2/ps3_CMBS4_norelicfast_beta2bestfit/inv_fisher.mat", skiprows=1)

#yesstep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/euclid/forecast_step_on/inv_fisher.mat", skiprows=1)
#nostep_covariance_in = np.loadtxt("/Users/nicholasdeporzio/Desktop/euclid/forecast_step_off/inv_fisher.mat", skiprows=1)

yesstep_covariance_out = np.zeros((11,11))
nostep_covariance_out = np.zeros((11,11))

for idx1 in range(11): 
    for idx2 in range(11): 
        yesstep_covariance_out[idx1, idx2] = yesstep_covariance_in[fisher_reorder_index[idx1], fisher_reorder_index[idx2]]
        nostep_covariance_out[idx1, idx2] = nostep_covariance_in[fisher_reorder_index[idx1], fisher_reorder_index[idx2]]

In [ ]:
# Specify fisher fiducials. Note omega_b in units of 10^2 and A_s in units of 10^-9. 
#fisher_fiducials = [2.222, 0.1120, 0.965, 2.22, 0.7, 0.06, 0.03, 250, 1.0, 1.0]

# DESI best fits
#mcmc_yesstep_bestfit = [2.225896, 0.1127521, 0.9659999, 2.234124, 0.7003915, 0.06018715, 0.0365804, 249.5779, 0.9992126, 1.025932]
#mcmc_nostep_bestfit = [2.226660, 0.1126965, 0.9662029, 2.231027, 0.7008683, 0.05944537, 0.0364634, 249.6839, 1.004659, 0.9794683]

#forecast_yesstep_shift = [
#    (2.226660e-00 + 1.092514e-05), 
#    (1.126965e-01 + 2.240456e-05),
#    (9.662029e-01 - 5.847986e-05),
#    (2.231027e-00 - 6.483268e-03),
#    (7.008683e-01 + 7.350068e-04),
#    (5.944537e-02 - 1.525700e-03),
#    (0.0364634 - 0.002999569333),
#    (2.496839e+02 + 9.316495e-02),
#    (1.004659e+00 - 4.432180e-03),
#    (9.794683e-01 + 3.124481e-02)
#    ]

# EUCLID best fits beta3
#mcmc_yesstep_bestfit = [ 2.225114e-02, 1.126566e-01, 9.668108e-01, 2.232845e-09, 7.014450e-01, 6.011749e-02, 3.492501e-02, 2.500968e+02, 1.699484e+00, 9.964380e-01]
#mcmc_nostep_bestfit = [2.225199e-02, 1.125597e-01, 9.669822e-01, 2.242754e-09, 7.005455e-01, 6.233920e-02, 3.891306e-02, 2.497794e+02, 1.707003e+00, 9.945805e-01]

# EUCLID best fits beta2
mcmc_yesstep_bestfit = [(2.225351e-02 * 1.0e2), 1.127173e-01, 9.669329e-01, (2.225880e-09 * 1.0e9), 7.011195e-01, 5.846847e-02, (1.035761e-01/3), 2.498382e+02, 1.701431e+00, 9.997678e-01, 9.844833e-01]
mcmc_nostep_bestfit = [(2.225588e-02 * 1.0e2), 1.126466e-01, 9.668334e-01, (2.238288e-09 * 1.0e9), 7.007553e-01, 6.127119e-02, (1.127821e-01/3), 2.498509e+02, 1.705295e+00, 1.002319e+00, 9.909839e-01]

#forecast_yesstep_shift = [
#    2.225199e-02+4.3676426956666724e-07, 
#    1.125597e-01+6.316268359389879e-05, 
#    9.669822e-01-0.00018290550762769392, 
#    2.242754e-09-4.9162108805243954e-12, 
#    7.005455e-01+0.00013939754967761536, 
#    6.233920e-02-0.0012388325575472259, 
#    3.891306e-02-(0.004071280305990854/3), 
#    2.497794e+02+33.295078986026056, 
#    1.707003e+00+0.007180663574527457, 
#    9.945805e-01+0.0015607510488493403
#]

#paramnames = [r'$\omega_b$', r'$\omega_{cdm}$', r'$n_s$', r'$A_s$', r'$h$', r'$tau_{reio}$', r'$m_0$', r'$\sigma_{fog}$', r'$\bar{b0}$', r'$\alpha_{k2}$']
paramnames = [r'omega_b', r'omega_cdm', r'n_s', r'A_s', r'h', r'tau_reio', r'm_0', r'sigma_fog', r'beta_0', r'beta_1', r'alpha_k2']
yesstep_gauss = GaussianND(mcmc_yesstep_bestfit, yesstep_covariance_out, names=paramnames)
nostep_gauss = GaussianND(mcmc_nostep_bestfit, nostep_covariance_out, names=paramnames)

In [ ]:
h = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
h.settings.figure_legend_frame = False #Box around legend
h.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
#h.settings.scaling_factor = 0.3 #Scaling of font size
#h.settings.title_limit_fontsize = 14
h.settings.axes_fontsize=14
h.settings.axes_labelsize=20
h.settings.axis_tick_x_rotation=45
h.settings.legend_fontsize=20
#h.settings.axis_tick_y_rotation=90
#h.settings.colorbar_tick_rotation=90

h.triangle_plot(
    [yesstep_gauss, nostep_gauss], 
    filled=True,
    legend_labels=['Fisher, EUCLID, Relic/LCDM Steps ON', 'Fisher, EUCLID, RELIC/LCDM Steps OFF'])
plt.savefig("/Users/nicholasdeporzio/Desktop/only_fisher.png")

In [ ]:
i = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
i.settings.figure_legend_frame = False #Box around legend
i.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
#i.settings.scaling_factor = 0.3 #Scaling of font size
#i.settings.title_limit_fontsize = 14
i.settings.axes_fontsize=14
i.settings.axes_labelsize=20
i.settings.axis_tick_x_rotation=45
i.settings.legend_fontsize=20
#i.settings.axis_tick_y_rotation=90
#i.settings.colorbar_tick_rotation=90

paramnames = ['omega_b', 'omega_cdm', 'n_s', 'A_s', 'h', 'tau_reio', 'm_0', 'sigma_fog', 'beta_0', 'beta_1', 'alpha_k2']
yesstep_gauss = GaussianND(mcmc_yesstep_bestfit, yesstep_covariance_out, names=paramnames)
nostep_gauss = GaussianND(mcmc_nostep_bestfit, nostep_covariance_out, names=paramnames)

i.triangle_plot(
    [samples_yesstep, samples_nostep, yesstep_gauss, nostep_gauss], 
    #filled=False,
    #legend_labels=['MCMC, DESI, Relic/LCDM Steps ON', 'MCMC, DESI, Relic/LCDM Steps OFF', 'Fisher, DESI, Relic/LCDM Steps ON', 'Fisher, DESI, Relic/LCDM Steps OFF'],
    legend_labels=['MCMC, EUCLID, Relic/LCDM Steps ON', 'MCMC, EUCLID, Relic/LCDM Steps OFF', 'Fisher, EUCLID, Relic/LCDM Steps ON', 'Fisher, EUCLID, Relic/LCDM Steps OFF'],
    line_args=[
        {'color':'green'},
        {'color':'darkblue'},
        {'ls':'--', 'lw':2, 'color':'red'},
        {'ls':'--', 'lw':2, 'color':'grey'}], 
    contour_args=[
        {'color':'green', 'filled' : True},
        {'color':'darkblue', 'filled' : True},
        {'ls':'--', 'lw':2, 'color':'red', 'filled' : False},
        {'ls':'--', 'lw':2, 'color':'grey', 'filled' : False}]
)

plt.savefig("/Users/nicholasdeporzio/Desktop/mcmc_and_fisher.png")

In [ ]:
# Show plots inline, and load main getdist plot module and samples class
from __future__ import print_function
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import sys, os
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
from getdist import plots, MCSamples, loadMCSamples
from getdist.gaussian_mixtures import GaussianND
import getdist
# use this *after* importing getdist if you want to use interactive plots
# %matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
import IPython
import numpy as np
import pandas as pd
print('GetDist Version: %s, Matplotlib version: %s'%(getdist.__version__, plt.matplotlib.__version__))
# matplotlib 2 may not work very well without usetex on, can uncomment
plt.rcParams['text.usetex']=True

# Full parameter set
#paramnames = [r'$\omega_b$', r'$\omega_{cdm}$', r'$n_s$', r'$A_s$', r'$\tau_{reio}$', 'h', r'$M_\chi$', r'$\sigma_{fog}$', r'$\bar{b}_0$', r'$\alpha_{k2}$']
#fiducials = [2.222, 0.1120, 0.965, 2.22, 0.06, 0.7, 0.09, 250, 1.0, 1.0]

# Reduced parameter set
paramnames = [r'$\omega_b$', r'$\omega_{cdm}$', r'$n_s$', r'$10^{9}A_s$', r'$\tau_{reio}$', '$h$', r'$M_\chi$']
fiducials = [2.222, 0.1120, 0.965, 2.22, 0.06, 0.7, 0.09]

reliccovmat = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/old_results/relicneutrino_match/relic/inv_fullfisher.mat", skiprows=1)
neutrinocovmat = np.loadtxt("/Users/nicholasdeporzio/Desktop/cfworkspace/results/old_results/relicneutrino_match/neutrino/inv_fullfisher.mat", skiprows=1)

relicgauss = GaussianND(fiducials, reliccovmat[0:-3, 0:-3], names=paramnames)
neutrinogauss = GaussianND(fiducials, neutrinocovmat[0:-3, 0:-3], names=paramnames)

g = plots.get_subplot_plotter()

# Set a default plot style 
from getdist.styles.tab10 import style_name
plots.set_active_style(style_name)

# Adjust plot sizes, styles, etc using getdist.plots.GetDistPlotSettings class
g.settings.figure_legend_frame = False #Box around legend
g.settings.scaling=False #Defaults to font size for Tick Labels: 11, Labels: 14, Legend: 12, Text: 12
#g.settings.scaling_factor = 0.3 #Scaling of font size
#g.settings.title_limit_fontsize = 14
g.settings.axes_fontsize=14
g.settings.axes_labelsize=20
g.settings.axis_tick_x_rotation=45
g.settings.legend_fontsize=20
#g.settings.axis_tick_y_rotation=90
#g.settings.colorbar_tick_rotation=90

g.triangle_plot(
    [relicgauss, neutrinogauss], 
    #filled=True,
    legend_labels=['Thermal Relic', 'Equivalent Neutrino'],
    line_args = [
        {'ls':'--', 'lw':2, 'color':'red'},
        {'ls':'--', 'lw':2, 'color':'grey'}],
    contour_args=[
        {'ls':'--', 'lw':2, 'color':'red', 'filled' : False},
        {'ls':'--', 'lw':2, 'color':'grey', 'filled' : False}])
plt.savefig("/Users/nicholasdeporzio/Desktop/relicneutrino_match.png")